# [Link to dataset](https://data.mendeley.com/datasets/zw4p9kj6nt/2)

In [1]:
%load_ext autotime

!nvidia-smi -L

import os

os.environ['CUDA_VISIBLE_DEVICES']='0'

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-3b49e2b8-87f0-c515-798b-3492ec05a183)
GPU 1: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-07628ed7-6ef8-fd67-7d03-cb6a89f72de4)


In [2]:
import numpy as np, tensorflow as tf, matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools, glob

# Experiment tracking with mlflow
import mlflow
import mlflow.tensorflow as mltf

In [3]:
tf.config.get_visible_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

```
mkdir mlflow && mlflow server \
    --backend-store-uri sqlite:///mlflow/mlflow.db \
    --default-artifact-root ./mlflow/artifacts \
    --host 0.0.0.0
```

In [4]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("mosquito")
mltf.autolog()

mlflow.set_tags({"Pretrain Model": "VGG16", 
                "Preprocessing" : "Keras VGG16 Preprocessing",
                "Pretrained Used Layers" : "first 13 Layers - Block4",
                "Framework": "tensorflow.keras"})

In [5]:
train_path = "./dataset/data_splitting/Train/"
valid_path = "./dataset/data_splitting/Test/"
test_path = "./dataset/data_splitting/Pred/"

In [6]:
mlflow.log_artifacts("./dataset/data_splitting/")

# How to save ImageDataGenerator parameters in mlflow ?

In [7]:
# You can add more augmentations, if you want

train_gen = ImageDataGenerator(
    rotation_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
)

gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [8]:
mlflow.log_params({"train ImageDataGenerator": {"rotation_range": 0.2,
                                                "horizontal_flip": True,
                                                "vertical_flip": True,
                                                "preprocessing_function": preprocess_input},
                   "test and valid ImageDataGenerator":{"preprocessing_function": preprocess_input}})

In [9]:
targetMap='''aegypti landing
aegypti smashed
albopictus landing
albopictus smashed
Culex landing
Culex smashed'''.split('\n')
targetMap

['aegypti landing',
 'aegypti smashed',
 'albopictus landing',
 'albopictus smashed',
 'Culex landing',
 'Culex smashed']

In [10]:
# Hyper-Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
NUM_CLASSES = len(targetMap)

In [11]:
mlflow.log_params({"IMG_SIZE":(224, 224), "Format":"RGB", "BATCH_SIZE": 32, "EPOCHS": 1,
                   "NUM_CLASSES": len(targetMap)})

In [12]:
train = train_gen.flow_from_directory(train_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)
valid = gen.flow_from_directory(valid_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)
test = gen.flow_from_directory(test_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)

Found 4200 images belonging to 6 classes.
Found 1799 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.


In [13]:
# Pretrained Model

ptm = VGG16(input_shape=list(IMG_SIZE)+[3], weights='imagenet', include_top=False)
ptm.trainable = False


vgg = Model(ptm.layers[0].input, ptm.layers[13].output)
vgg.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
# save image as artifact

# keras.utils.plot_model(vgg, to_file='pretrainedModel.png')
# mlflow.log_artifact("pretrainedModel.png")

In [15]:
# But I prefer a text file

run_id = mlflow.active_run().info.run_id # same as uuid

def summary_to_text(summay):
    with open(f"./mlflow/artifacts/1/{run_id}/pretrained_summary.txt", "w") as f:
        print(summay, file=f)
        
vgg.summary(print_fn=summary_to_text)

mlflow.log_artifact(f"./mlflow/artifacts/1/{run_id}/pretrained_summary.txt")

In [16]:
i = layers.Input(shape=(IMG_SIZE[:]+(3,)))
p = layers.Rescaling(scale=1./255.)(i)
p = vgg(p)
x = layers.BatchNormalization()(p)
x = layers.Conv2D(256, (3, 3), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2,2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2,2))(x)
x = layers.Dropout(0.1)(x)
x= layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128)(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64)(x)
x= layers.Dense(NUM_CLASSES, activation='linear')(x)

model = Model(i, x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 model (Functional)          (None, 28, 28, 512)       7635264   
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 512)      2048      
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 256)       1179904   
                                                                 
 batch_normalization_1 (Batc  (None, 26, 26, 256)      1024      
 hNormalization)                                           

In [17]:
model.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['acc'])

h = model.fit(train, validation_data=valid, epochs=EPOCHS)

Epoch 1/25
132/132 [==============================] - 43s 293ms/step - loss: 2.6374 - acc: 0.5545 - val_loss: 0.7401 - val_acc: 0.7065
Epoch 2/25
132/132 [==============================] - 36s 271ms/step - loss: 0.7386 - acc: 0.7383 - val_loss: 0.4694 - val_acc: 0.8255
Epoch 3/25
132/132 [==============================] - 36s 269ms/step - loss: 0.5445 - acc: 0.8086 - val_loss: 0.6848 - val_acc: 0.7465
Epoch 4/25
132/132 [==============================] - 36s 273ms/step - loss: 0.4791 - acc: 0.8207 - val_loss: 0.3630 - val_acc: 0.8688
Epoch 5/25
132/132 [==============================] - 36s 270ms/step - loss: 0.3680 - acc: 0.8705 - val_loss: 0.6120 - val_acc: 0.7888
Epoch 6/25
132/132 [==============================] - 36s 269ms/step - loss: 0.3281 - acc: 0.8821 - val_loss: 0.4532 - val_acc: 0.8477
Epoch 7/25
132/132 [==============================] - 36s 272ms/step - loss: 0.2816 - acc: 0.8960 - val_loss: 0.2485 - val_acc: 0.9122
Epoch 8/25
132/132 [==============================] - 3

INFO:tensorflow:Assets written to: /tmp/tmpiv8mbfna/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpiv8mbfna/model/data/model/assets
2022/08/18 00:38:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpiv8mbfna/model, flavor: keras), fall back to return ['tensorflow==2.9.0', 'keras==2.9.0']. Set logging level to DEBUG to see the full traceback.


In [19]:
test_loss, test_acc = model.evaluate(test)

mlflow.log_metrics({
    "test loss": test_loss,
    "test accuracy": test_acc
})

113/113 [==============================] - 5s 45ms/step - loss: 0.2003 - acc: 0.9467
